## OpenStreetMap Overpass API

### Overpass API の概要

- Overpass API は、OpenStreetMap のデータを取得するための読み取り専用の API です
  - https://overpass-turbo.eu/
  - https://wiki.openstreetmap.org/wiki/JA:Overpass_API
- Overpass API は、 XML または Overpass QL という専用のクエリ言語を使用してデータを取得します
  - https://wiki.openstreetmap.org/wiki/JA:Overpass_API/Overpass_QL

### 注意

- このノートブックでは、 z.overpass-api.de で提供されている Overpass API を使用しています
- z.overpass-api.de で提供されている Overpass API は、 **一日あたり10,000クエリまたは1GBのデータ取得** が目安です
  - この頻度を超えるリクエストが必要な場合は、自分で Overpass API サーバーを立てることができます
  - インストール方法は公式ドキュメントを参照してください
    - https://overpass-api.de/no_frills.html
    - https://overpass-api.de/full_installation.html
    - https://wiki.openstreetmap.org/wiki/Overpass_API/Installation


### このノートブックの流れ

- 生成 AI に、自然言語から Overpass QL を出力させる
- 出力された Overpass QL で、 Overpass API からデータを取得する
- 取得したデータを GeoJSON に変換する
- GeoJSON を地図上に表示する

### 前準備

### 対象となる自然言語

In [1]:
input_text = "長崎県長崎市のカフェを地図に表示してください。"
print(input_text)

長崎県長崎市のカフェを地図に表示してください。


### 生成 AI に自然言語から Overpass QL を出力させる

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# モデルの準備
model = ChatGoogleGenerativeAI(model="gemini-exp-1206", temperature=0)

# プロンプトの準備
template = """You are an expert in Overpass API Query.
Output the appropriate Overpass API Query from the user input.

You will always reply according to the following rules:
- Output valid Overpass API query.
- The query MUST be out json.
- The query MUST be out geom.
- The query MUST be set timeout as 30000.
- The query will utilize a area specifier as needed.
- The query will search nwr as needed.
- The query MUST be line delimited and surrounded by just three backquote to indicate that it is a code block.

Hints:
- Regardless of the language of the user input, use area["name"="..."]. Because the area names are in the local language.
- If the user input is for a specific type of restaurant, amenity should be restaurant and filter by cuisine.

** Important **
Take a deep breath and carefully check if it is in accordance with the rules and appropriate as an Overpass API Query.

User Input:
{input}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

res = chain.invoke({"input": input_text})
result = res.content.strip()
print(result)

```
[out:json][timeout:30000];
area["name"="長崎市"]->.searchArea;
(
  nwr["amenity"="cafe"](area.searchArea);
);
out geom;
```


In [7]:
# resultから ``` を使って Overpass QL のみを抽出

import re

# 正規表現でコードブロックの中身を抽出
match = re.search(r"```[^\n]*\n(.*?)```", result, re.DOTALL)

if match:
    overpass_query = match.group(1).strip()
    print(overpass_query)
else:
    print("Overpass API Queryが見つかりませんでした。")

[out:json][timeout:30000];
area["name"="長崎市"]->.searchArea;
(
  nwr["amenity"="cafe"](area.searchArea);
);
out geom;


### 出力された Overpass QL で Overpass API からデータを取得する

In [8]:
import requests

def get_overpass_response(overpass_query: str):
    query_string = f"data={requests.utils.quote(overpass_query)}"
    overpass_api_url = f"https://z.overpass-api.de/api/interpreter?{query_string}"
    response = requests.get(overpass_api_url)
    if response.status_code == 200:
        res_json = response.json()
        return res_json
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

In [9]:
import json

overpass_response = get_overpass_response(overpass_query)
print(json.dumps(overpass_response, indent=2, ensure_ascii=False))
print(len(overpass_response['elements']))

{
  "version": 0.6,
  "generator": "Overpass API 0.7.62.5 1bd436f1",
  "osm3s": {
    "timestamp_osm_base": "2025-03-08T10:15:36Z",
    "timestamp_areas_base": "2024-12-31T00:49:33Z",
    "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL."
  },
  "elements": [
    {
      "type": "node",
      "id": 2503809124,
      "lat": 32.7360454,
      "lon": 129.8703754,
      "tags": {
        "amenity": "cafe",
        "layer": "1",
        "level": "1",
        "name": "タイム",
        "source": "image,2013-09;Bing,2011-07-11"
      }
    },
    {
      "type": "node",
      "id": 2503853734,
      "lat": 32.7359147,
      "lon": 129.8695208,
      "tags": {
        "amenity": "cafe",
        "name": "BRICK CAFE",
        "source": "image,2013-09;Bing,2011-07-11"
      }
    },
    {
      "type": "node",
      "id": 2503894610,
      "lat": 32.7357577,
      "lon": 129.8698889,
      "tags": {
        "amenity": "cafe",
      

### Overpass API で取得したデータを GeoJSON に変換する

In [10]:
def convert_overpass_to_geojson(overpass_response):
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for element in overpass_response['elements']:
        feature = {
            "type": "Feature",
            "properties": {},
            "geometry": {}
        }

        if 'tags' in element:
            feature['properties'] = element['tags']

        if element['type'] == 'node':
            feature['geometry'] = {
                "type": "Point",
                "coordinates": [element['lon'], element['lat']]
            }

        elif element['type'] == 'way':
            feature['geometry'] = {
                "type": "Polygon" if 'nodes' in element else "LineString",
                "coordinates": [[(n['lon'], n['lat']) for n in overpass_response['elements'] if n['id'] in element['nodes']]]
            }

        elif element['type'] == 'relation':
            # Handle relations as MultiPolygons (simplified)
            feature['geometry'] = {"type": "MultiPolygon", "coordinates": []}
            for member in element['members']:
                if member['type'] == 'way':
                    way_id = member['ref']
                    way = next((w for w in overpass_response['elements'] if w['id'] == way_id and w['type'] == 'way'), None)
                    if way:
                        coordinates = [[(n['lon'], n['lat']) for n in overpass_response['elements'] if n['id'] in way['nodes']]]
                        if coordinates:
                            feature['geometry']['coordinates'].append([coordinates[0]]) #Multipolygonなので、さらにリストで囲む

        geojson["features"].append(feature)
    return geojson

In [11]:

geojson_data = convert_overpass_to_geojson(overpass_response)
print(json.dumps(geojson_data, indent=2, ensure_ascii=False))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "amenity": "cafe",
        "layer": "1",
        "level": "1",
        "name": "タイム",
        "source": "image,2013-09;Bing,2011-07-11"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          129.8703754,
          32.7360454
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "amenity": "cafe",
        "name": "BRICK CAFE",
        "source": "image,2013-09;Bing,2011-07-11"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          129.8695208,
          32.7359147
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "amenity": "cafe",
        "name": "トレビ",
        "name:en": "TOREVI",
        "source": "image,2013-09;Bing,2011-07-11"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          129.8698889,
          32.7

### GeoJSON を地図上に表示する

In [12]:
import folium

m = folium.Map(location=[0, 0], zoom_start=2)

folium.GeoJson(geojson_data).add_to(m)

bounds = folium.GeoJson(geojson_data).get_bounds()
if bounds:
    m.fit_bounds(bounds)

display(m)